In [1]:
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile

def save_graph_to_file(sess,  graph_file_name, output_names):
    output_graph_def = graph_util.convert_variables_to_constants(
      sess,  sess.graph.as_graph_def(),  output_names)
    with gfile.FastGFile(graph_file_name, 'wb') as f:
        f.write(output_graph_def.SerializeToString())

def keras2tflite(sess, input_tensors, output_tensors, filename):
    '''
    input_tensor and output_tensors is array and is keras input and output tensor
    filename must be .tflite
    '''
    converter = tf.contrib.lite.TocoConverter.from_session(sess, input_tensors, output_tensors)
    tflite_model=converter.convert()
    open(filename, "wb").write(tflite_model)

In [21]:
from keras.models import Sequential, Model
from keras.models import load_model
import tensorflow as tf
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras import optimizers
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

train_path='/home/sober/code/jupyter/ftp_code/AndroidTensorflow/tensorflow-for-poets-2/tf_files/flower_photos'
val_path='/home/sober/code/jupyter/ftp_code/AndroidTensorflow/tensorflow-for-poets-2/tf_files/flower_photos'
batch_size = 16
epochs = 100
def mean(x):
    return (x-128.0)

input = Input(shape = (150, 150, 3), name = 'input')
x = Conv2D(32, (3, 3), activation='relu')(input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
# x = Dense(64, activation='relu')(x)
# x = Dense(2, activation='sigmoid', name = 'output')(x)
x = Dense(5, activation='softmax', name = 'output')(x)
model = Model(inputs = input, outputs= x)

sgd = optimizers.SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True, preprocessing_function=mean)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(preprocessing_function=mean)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        val_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch=1340 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        shuffle=True,
        validation_steps=64 // batch_size)
model.save('./weight/flower5tlite_mean.h5')  # always save your weights after training or during training

# convert
output_names = [node.op.name for node in model.outputs]

export_dir = './weight/'
sess = K.get_session()
# save_graph_to_file(sess,  export_dir + "flower5.pb", output_names)
keras2tflite(sess, [input], [x], export_dir + "flower5tlite_mean.tflite")


Found 3670 images belonging to 5 classes.
Found 3670 images belonging to 5 classes.
Epoch 1/100
83/83 [==============================] - 10s 126ms/step - loss: 1.5463 - acc: 0.3313 - val_loss: 1.2532 - val_acc: 0.3750
Epoch 2/100
83/83 [==============================] - 10s 122ms/step - loss: 1.2651 - acc: 0.4511 - val_loss: 1.4297 - val_acc: 0.3750
Epoch 3/100
83/83 [==============================] - 10s 119ms/step - loss: 1.2330 - acc: 0.4659 - val_loss: 1.2457 - val_acc: 0.4844
Epoch 4/100
83/83 [==============================] - 11s 127ms/step - loss: 1.1470 - acc: 0.5113 - val_loss: 1.2128 - val_acc: 0.4219
Epoch 5/100
83/83 [==============================] - 10s 124ms/step - loss: 1.1165 - acc: 0.5233 - val_loss: 0.9825 - val_acc: 0.5781
Epoch 6/100
83/83 [==============================] - 10s 117ms/step - loss: 1.1084 - acc: 0.5524 - val_loss: 1.1478 - val_acc: 0.5312
Epoch 7/100
83/83 [==============================] - 10s 123ms/step - loss: 1.0826 - acc: 0.5512 - val_loss: 1.1

83/83 [==============================] - 10s 117ms/step - loss: 0.5352 - acc: 0.8062 - val_loss: 0.4720 - val_acc: 0.8125
Epoch 62/100
83/83 [==============================] - 10s 118ms/step - loss: 0.5115 - acc: 0.8117 - val_loss: 0.4345 - val_acc: 0.8750
Epoch 63/100
83/83 [==============================] - 10s 121ms/step - loss: 0.5489 - acc: 0.7967 - val_loss: 0.3844 - val_acc: 0.8594
Epoch 64/100
83/83 [==============================] - 10s 120ms/step - loss: 0.5308 - acc: 0.7999 - val_loss: 0.2737 - val_acc: 0.9219
Epoch 65/100
83/83 [==============================] - 10s 126ms/step - loss: 0.5180 - acc: 0.8050 - val_loss: 0.4563 - val_acc: 0.8281
Epoch 66/100
83/83 [==============================] - 10s 120ms/step - loss: 0.5073 - acc: 0.8050 - val_loss: 0.4615 - val_acc: 0.7969
Epoch 67/100
83/83 [==============================] - 10s 116ms/step - loss: 0.5196 - acc: 0.8042 - val_loss: 0.4303 - val_acc: 0.8750
Epoch 68/100
83/83 [==============================] - 10s 126ms/step